In [ ]:
import pandas as pd
import plotly.express as px

# Cargar la base de datos
ruta_archivo = 'BASE DE DATOS DE CONCIERTOS.xlsx'
df = pd.read_excel(ruta_archivo, sheet_name='Hoja1')

# Limpiar datos
df['ENTRADA_MÁS_BARATA'] = pd.to_numeric(df['ENTRADA_MÁS_BARATA'], errors='coerce')
df['ENTRADA_MÁS_CARA'] = pd.to_numeric(df['ENTRADA_MÁS_CARA'], errors='coerce')
df['AÑO'] = pd.to_datetime(df['AÑO'], format='%Y')
df['MERCADO'] = df['MERCADO'].str.strip().str.lower()

# Agregar datos
df_agregado_barata = df.groupby(['MERCADO', 'AÑO'])['ENTRADA_MÁS_BARATA'].mean().reset_index()
df_agregado_cara = df.groupby(['MERCADO', 'AÑO'])['ENTRADA_MÁS_CARA'].mean().reset_index()

# Crear pivots
pivot_barata = df_agregado_barata.pivot(index='MERCADO', columns='AÑO', values='ENTRADA_MÁS_BARATA')
pivot_cara = df_agregado_cara.pivot(index='MERCADO', columns='AÑO', values='ENTRADA_MÁS_CARA')

# Definir paleta personalizada
paleta_personalizada = ["#d460ff", "#a193f3", "#d6a4e9", "#e667b1"]

# Determinar umbrales para eliminar el tono rosado (ajustamos por percentil 90 o 95)
umbral_barata = pivot_barata.quantile(0.95).max()
umbral_cara = pivot_cara.quantile(0.95).max()

# Reemplazar valores extremos con NaN
pivot_barata_filtrado = pivot_barata.mask(pivot_barata >= umbral_barata)
pivot_cara_filtrado = pivot_cara.mask(pivot_cara >= umbral_cara)

# Visualizar sin los valores más altos
fig_barata = px.imshow(
    pivot_barata_filtrado,
    labels=dict(x="Año", y="Mercado", color="Precio"),
    title="Mapa sin precios más altos (entrada más barata)",
    color_continuous_scale=paleta_personalizada
)

fig_cara = px.imshow(
    pivot_cara_filtrado,
    labels=dict(x="Año", y="Mercado", color="Precio"),
    title="Mapa sin precios más altos (entrada más cara)",
    color_continuous_scale=paleta_personalizada
)

fig_barata.show()
fig_cara.show()

# Mostrar los valores que fueron excluidos
print("🔍 Precios más altos excluidos (barata):", df_agregado_barata['ENTRADA_MÁS_BARATA'].max())
print("🔍 Precios más altos excluidos (cara):", df_agregado_cara['ENTRADA_MÁS_CARA'].max())

🔍 Precios más altos excluidos (barata): 186.75
🔍 Precios más altos excluidos (cara): 492.87272727272733


In [ ]:
# Guardar visualizaciones enn HTML
fig_barata.write_html("mapa_calor_barata.html")
fig_cara.write_html("mapa_calor_cara.html")